# Chronos Bolt Base - 0 Shot, No covariates

Document: https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#zero-shot-forecasting

In [28]:
import datetime

import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.metrics import root_mean_squared_error

In [15]:
# Experiment設定
EXPERIMENT_NAME = 'ChronosBolt_0shot'
MODEL = "bolt_base"
FORECAST_HORIZON = 6

In [2]:
!ls ../data

walmart_ts_6_fcst_grp_test.csv  walmart_ts_6_fcst_grp_train.csv


In [3]:
df_train = pd.read_csv('../data/walmart_ts_6_fcst_grp_train.csv')
df_test = pd.read_csv('../data/walmart_ts_6_fcst_grp_test.csv')
print(df_train.shape, df_test.shape)
#df_train.head()

# 日付カラムを型変換する場合
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test['Date'] = pd.to_datetime(df_test['Date'])

# 単一のグループIDを作成
df_train['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_train['Store'], df_train['Dept'])]
df_test['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_test['Store'], df_test['Dept'])]

# 利用カラムのみを選択
df_train = df_train[['Store_Dept','Date','Weekly_Sales']]
df_test = df_test[['Store_Dept','Date','Weekly_Sales']]
print(df_train.shape, df_test.shape)
df_train.head()

(702, 11) (36, 11)
(702, 3) (36, 3)


,Store_Dept,Date,Weekly_Sales
0,4_4,2010/2/5,59554.57
1,4_4,2010/2/12,54069.82
2,4_4,2010/2/19,53939.17
3,4_4,2010/2/26,54687.08
4,4_4,2010/3/5,56959.02


In [20]:
# TimeSeriesDataFrameへ変換
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column='Store_Dept',
    timestamp_column="Date"
)
print(type(train_data))
train_data.head()

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>


Weekly_Sales
item_id timestamp               
4_4     2010-02-05      59554.57
        2010-02-12      54069.82
        2010-02-19      53939.17
        2010-02-26      54687.08
        2010-03-05      56959.02

In [22]:
%%time

log_dir = EXPERIMENT_NAME +'_'+ datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
print(log_dir)

predictor = TimeSeriesPredictor(
    target="Weekly_Sales",
    eval_metric="RMSE",
    prediction_length=FORECAST_HORIZON,  # 予測期間の長さ
    path="./tmp/"+log_dir,
).fit(
    train_data, presets=MODEL,
)
predictor

Beginning AutoGluon training...
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_0shot_2025-02-27_09h46m05s'


ChronosBolt_0shot_2025-02-27_09h46m05s


=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.99 GB / 32.00 GB (46.8%)
Disk Space Avail:   1437.16 GB / 1863.39 GB (77.1%)
Setting presets to: bolt_base

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSE,
 'hyperparameters': {'Chronos': {'model_path': 'bolt_base'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'Weekly_Sales',
 'verbosity': 2}

Inferred time series frequency: 'W-FRI'
Provided train_data has 702 rows, 6 time series. Median time series length is 117

CPU times: user 2.74 s, sys: 618 ms, total: 3.35 s
Wall time: 3.66 s


In [23]:
# 与えられたデータの最後の日付から、prediction_length分を予測
predictions = predictor.predict(train_data)
predictions

Model not specified in predict, will default to the model with the best validation score: Chronos[bolt_base]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  58457.667969  54596.765625  55801.671875  56712.921875   
        2012-05-11  59094.867188  55017.210938  56232.050781  57214.839844   
        2012-05-18  59522.289062  55317.265625  56584.996094  57616.019531   
        2012-05-25  59884.371094  55599.445312  56930.289062  57973.109375   
        2012-06-01  60301.242188  55800.687500  57216.703125  58309.281250   
        2012-06-08  59724.187500  55307.351562  56713.015625  57790.410156   
8_13    2012-05-04  24584.298828  22936.460938  23514.357422  23921.964844   
        2012-05-11  24573.091797  22800.037109  23409.107422  23852.595703   
        2012-05-18  24623.982422  22806.966797  23438.673828  23893.103516   
        2012-05-25  24609.269531  22783.496094  23426.009766  23884.355469   
        2012-06-01  24641.873047  22782.935547  23436.654297  23903.726562   
        2012-06-08  24654.166016  22786.335938  23446.931641  23914.947266   
1_13    2012-05-04  40660.468750  38148.261719  39043.625000  39646.757812   
        2012-05-11  40801.937500  38080.742188  39023.921875  39691.996094   
        2012-05-18  40048.816406  37279.648438  38246.855469  38938.906250   
        2012-05-25  40197.464844  37277.195312  38303.375000  39022.078125   
        2012-06-01  41434.429688  38205.195312  39324.738281  40122.296875   
        2012-06-08  41794.644531  38475.359375  39650.953125  40451.875000   
13_8    2012-05-04  35158.121094  33065.687500  33761.683594  34280.539062   
        2012-05-11  35119.320312  32921.242188  33645.949219  34193.187500   
        2012-05-18  35015.089844  32790.777344  33540.175781  34098.332031   
        2012-05-25  34987.808594  32727.343750  33495.675781  34063.171875   
        2012-06-01  34907.308594  32602.337891  33393.570312  33971.941406   
        2012-06-08  34827.671875  32499.386719  33299.054688  33887.519531   
19_8    2012-05-04  41763.679688  39193.328125  40048.335938  40681.218750   
        2012-05-11  41760.894531  39048.132812  39927.871094  40600.640625   
        2012-05-18  41688.566406  38948.796875  39844.804688  40527.828125   
        2012-05-25  41722.265625  38947.960938  39868.320312  40555.625000   
        2012-06-01  41770.781250  38944.285156  39886.101562  40587.375000   
        2012-06-08  41812.421875  38965.375000  39917.917969  40624.347656   
31_13   2012-05-04  43324.082031  40802.539062  41640.300781  42255.679688   
        2012-05-11  43922.718750  41103.277344  41998.816406  42693.449219   
        2012-05-18  44781.820312  41763.222656  42738.914062  43487.273438   
        2012-05-25  44525.191406  41565.949219  42543.367188  43274.996094   
        2012-06-01  44184.402344  41205.601562  42208.621094  42947.445312   
        2012-06-08  44214.035156  41162.542969  42198.878906  42960.218750   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  57568.484375  58457.667969  59402.519531  60462.789062   
        2012-05-11  58136.042969  59094.867188  60093.238281  61203.996094   
        2012-05-18  58572.550781  59522.289062  60523.183594  61650.089844   
        2012-05-25  58930.605469  59884.371094  60892.179688  62031.269531   
        2012-06-01  59301.988281  60301.242188  61360.632812  62529.566406   
        2012-06-08  58742.230469  59724.187500  60751.621094  61938.222656   
8_13    2012-05-04  24266.589844  24584.298828  24907.386719  25273.328125   
        2012-05-11  24228.335938  24573.091797  24919.478516  25310.703125   
        2012-05-18  24276.699219  24623.982422  24974.130859  25368.857422   
        2012-05-25  24265.251953  24609.269531  24960.644531  25357.302734   
        2012-06-01  24287.730469  24641.873047  24994.904297  25394.523438   
        2012-06-08  24301.464844  24654.166016  2

In [32]:
df_pred = predictions.to_data_frame().reset_index()

print('RMSE: ', root_mean_squared_error(y_true=df_test['Weekly_Sales'], y_pred=df_pred['mean']))

df_pred.to_csv('../Chronos_results/{}.csv'.format(EXPERIMENT_NAME), index=False)

RMSE:  1710.3089552947943
